In [1]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SC201L16


In [2]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [3]:
# Load Existing Dataset
import torchvision.datasets as dset
import torchvision.transforms as T

train_data = dset.MNIST('./train',train=True,download=True)
val_data = dset.MNIST('./val',train=False,download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 132708911.20it/s]


Extracting ./train/MNIST/raw/train-images-idx3-ubyte.gz to ./train/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 148087645.26it/s]


Extracting ./train/MNIST/raw/train-labels-idx1-ubyte.gz to ./train/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 15382526.00it/s]


...

Extracting ./val/MNIST/raw/train-labels-idx1-ubyte.gz to ./val/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 38077441.11it/s]


Extracting ./val/MNIST/raw/t10k-images-idx3-ubyte.gz to ./val/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4937928.66it/s]


Extracting ./val/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./val/MNIST/raw



In [7]:
print(train_data)
train_data[0][0]

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train


In [ ]:
# Check Data Dimension
#####################################
#                                   #
#               TODO:               #
#                                   #
#####################################

In [ ]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

In [ ]:
# Build Model
import torch.nn as nn
model = None
#####################################
#                                   #
#               TODO:               #
#                                   #
#####################################

In [ ]:
# Move model to GPU
#####################################
#                                   #
#               TODO:               #
#                                   #
#####################################

In [ ]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [ ]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [ ]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [ ]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)